# Quantifying the Impact of Sampling Error in Social Vulnerability Index Construction

### Downloading VRT

In [ ]:
from data_processing import download_replicates
from data_processing import combine_states
import os

path = os.path.join(os.path.dirname(os.getcwd()))
ipath = os.path.join(path,'0_data', 'input')

with open(os.path.join(ipath, 'tables_new.txt'), 'r') as f:
    table_names = f.read().splitlines()

download_replicates(table_names)
combine_states() 

#### Build Denominator

In [ ]:

from replicate_indicators import calculate_denominators
from data_processing import load_replicates
from csv_file import save_to_csv

dataframes, table_names = load_replicates()
for name, df in dataframes.items():
    exec(f"{name} = df")

df_denominators = calculate_denominators(dataframes)
save_to_csv(df_denominators, 'gitignore', 'df_denominators.csv')  

#### Build Indicator

In [ ]:
from replicate_indicators import calculate_indicators
from csv_file import save_to_csv

df_indicators = calculate_indicators(dataframes)
save_to_csv(df_indicators, 'gitignore', 'df_indicators.csv')

#### Calculate Nominal Index and MOE of Index

In [ ]:
from csv_file import read_from_csv

df_indicators = read_from_csv('df_indicators_continental.csv', 'gitignore/filtered_datasets')
df_denominators = read_from_csv('df_denominators_continental.csv', 'gitignore/filtered_datasets')
n_indicators = len(df_indicators)
n_denominators = len(df_denominators)
print(f"Number of rows in df_indicators: {n_indicators}")
print(f"Number of rows in df_denominators: {n_denominators}")

# Check how many GEOIDs are not matching
ind_geoids = set(df_indicators['GEOID'])
den_geoids = set(df_denominators['GEOID'])
only_in_ind = ind_geoids - den_geoids
only_in_den = den_geoids - ind_geoids
print(f"GEOIDs only in df_indicators: {len(only_in_ind)}")
print(f"GEOIDs only in df_denominators: {len(only_in_den)}")

In [ ]:
from calculate_denomination import estimate_denomination
from nominal_index import construct_nominal_index
from csv_file import save_to_csv, read_from_csv

df_indicators = read_from_csv('df_indicators_continental.csv', 'gitignore/filtered_datasets')
df_denominators = read_from_csv('df_denominators_continental.csv', 'gitignore/filtered_datasets')

# denominate indicators based on Estimate of indicator and denominator
df_variables = estimate_denomination(df_indicators, df_denominators)

# nominal index construction different configurations
nominal_index = construct_nominal_index(df_variables, 'minmax', False, 'arithmetic', 'equal', None)
save_to_csv(nominal_index, 'gitignore', 'nominal_index_minmax_no_norm.csv')
nominal_index = construct_nominal_index(df_variables, 'minmax', True, 'arithmetic', 'equal', None)
save_to_csv(nominal_index, 'gitignore', 'nominal_index_minmax_norm.csv')
nominal_index = construct_nominal_index(df_variables, 'pct', False, 'arithmetic', 'equal', None)
save_to_csv(nominal_index , 'gitignore', 'nominal_index_pct_no_norm.csv')
nominal_index = construct_nominal_index(df_variables, 'pct', True, 'arithmetic', 'equal', None)
save_to_csv(nominal_index , 'gitignore', 'nominal_index_pct_norm.csv')
nominal_index = construct_nominal_index(df_variables, 'pct', True, 'multiplicative', 'equal', None)
save_to_csv(nominal_index , 'gitignore', 'nominal_index_pct_norm_mult_equ.csv')
nominal_index = construct_nominal_index(df_variables, 'minmax', True, 'multiplicative', 'equal', None)
save_to_csv(nominal_index , 'gitignore', 'nominal_index_minmax_norm_mult_equ.csv')
weights = {"THEME1": 0.37, "THEME2": 0.43, "THEME3": 0.18, "THEME4": 0.2}
nominal_index = construct_nominal_index(df_variables, 'pct', True, 'arithmetic', 'weights', weights)
save_to_csv(nominal_index , 'gitignore', 'nominal_index_pct_norm_arith_weights.csv')

### Manuscript Results - Section One Moe

Geometric Aggregation

In [ ]:
from calculate_denomination import estimate_denomination
from nominal_index import construct_nominal_index
from csv_file import read_from_csv
from moe_stability import plot_moe_distribution_per_bin
from replicate_indices import construct_replicate_indices
from csv_file import read_from_csv
from moe_stability import class_stability, plot_moe
import warnings
warnings.filterwarnings('ignore')

df_indicators = read_from_csv('df_indicators_continental.csv', 'gitignore/filtered_datasets')
df_denominators = read_from_csv('df_denominators_continental.csv', 'gitignore/filtered_datasets')

# denominate 
df_variables = estimate_denomination(df_indicators, df_denominators)
initial_rows = len(df_variables)
df_variables = df_variables.merge(df_denominators[['GEOID', 'E_TOTHH']], on='GEOID', how='left')
df_variables = df_variables[df_variables['E_TOTHH'] >= 30].copy()
df_variables = df_variables.drop(columns=['E_TOTHH'])
final_rows = len(df_variables)
rows_removed = initial_rows - final_rows
print(f"Number of rows removed: {rows_removed}")

# Construct nominal index 
nominal_index = construct_nominal_index(df_variables, 'minmax', True, 'geometric', 'equal', None)
print("nominal index finished")
df_nominal_index_pct = nominal_index.copy()

# VRT
var_rep_indices_pct = construct_replicate_indices(df_nominal_index_pct, 'minmax', True , 'geometric', 'equal', None)
df_index_moe = class_stability(var_rep_indices_pct)  

# Scatterplot
fig = plot_moe(df_index_moe, 40, 50)
name_fig = '0_geometric_moe_UPDATE'
fig.savefig(f"../2_results/gitignore/{name_fig}.tiff", dpi=600)
fig.savefig(f"../2_results/gitignore/{name_fig}.pdf")
fig.savefig(f"../2_results/gitignore/{name_fig}.svg", format='svg')

# Boxplot 
fig = plot_moe_distribution_per_bin(df_index_moe, plot_type='box',y_max=15, quantile_bins=True, n_quantiles=4)
name_fig = '0_geometric_box_quantile'
fig.savefig(f"../2_results/gitignore/{name_fig}.tiff", dpi=600)
fig.savefig(f"../2_results/gitignore/{name_fig}.pdf")
fig.savefig(f"../2_results/gitignore/{name_fig}.svg", format='svg')

#### Panel Plot

Data

In [ ]:
from replicate_indices import construct_replicate_indices
from csv_file import save_to_csv, read_from_csv

# A plot 
df_nominal_index_minmax = read_from_csv('nominal_index_minmax_no_norm.csv', 'gitignore')
var_rep_indices_minmax = construct_replicate_indices(df_nominal_index_minmax, 'minmax', False, 'arithmetic', 'equal', None)
save_to_csv(var_rep_indices_minmax, 'gitignore/panel_plot', 'var_rep_min_no_norm.csv')


# B plot
df_nominal_index_minmax = read_from_csv('nominal_index_minmax_norm.csv', 'gitignore')
var_rep_indices_minmax = construct_replicate_indices(df_nominal_index_minmax, 'minmax', True, 'arithmetic', 'equal', None)
save_to_csv(var_rep_indices_minmax, 'gitignore/panel_plot', 'var_rep_min_norm.csv')

# C plot
df_nominal_index_pct = read_from_csv('nominal_index_pct_no_norm.csv', 'gitignore')
var_rep_indices_pct = construct_replicate_indices(df_nominal_index_pct, 'pct', False, 'arithmetic', 'equal', None)
save_to_csv(var_rep_indices_pct, 'gitignore/panel_plot', 'var_rep_pct_no_norm.csv')

# D plot
df_nominal_index_pct = read_from_csv('nominal_index_pct_norm.csv', 'gitignore')
var_rep_indices_pct = construct_replicate_indices(df_nominal_index_pct, 'pct', True, 'arithmetic', 'equal', None)
save_to_csv(var_rep_indices_pct, 'gitignore/panel_plot', 'var_rep_pct_norm_arith_equ.csv')

Plot

In [ ]:
from csv_file import read_from_csv
import matplotlib.pyplot as plt
from moe_stability import class_stability
from panel_plot import plot_panel

# data
df_moe_dict = {
    'A': class_stability(read_from_csv('var_rep_min_no_norm.csv', '')),
    'B': class_stability(read_from_csv('var_rep_min_norm.csv', '')),
    'C': class_stability(read_from_csv('var_rep_pct_no_norm.csv', '')),
    'D': class_stability(read_from_csv('var_rep_pct_norm_arith_equ.csv', '')),
}

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()
fig.patch.set_facecolor('white')
for ax in axes:
    ax.set_facecolor('white')
labels = ['A', 'B', 'C', 'D']  
y_limits = [40, 40, 40, 40]    
mappables = []
for i, ax in enumerate(axes):
    label = labels[i]
    df = df_moe_dict[label]
    mesh = plot_panel(df, y_limits[i], ax=ax, show_labels=False)
    ax.set_xlim(0, 100)
    mappables.append(mesh)
    ax.text(0.06, 0.98, label, transform=ax.transAxes,
            fontsize=14, fontweight='bold', ha='left', va='top')
fig.text(0.5, 0.02, 'Social Vulnerability Index-Score', ha='center', fontsize=14)
fig.text(0.02, 0.5, 'Margin of Error of Vulnerability Index-Score', va='center',
         rotation='vertical', fontsize=14)

fig.subplots_adjust(left=0.08, right=0.92, top=0.96, bottom=0.08,
                    hspace=0.1, wspace=0.1)

fig.colorbar(mappables[-1], ax=axes, orientation='vertical', fraction=0.02, pad=0.02)
cbar = ax.figure.axes[-1]  
cbar.set_ylabel('Number of Census Tracts', fontsize= 14)
fig.savefig("../2_results/fig_3_panel.tiff", dpi=600)
fig.savefig("../2_results/fig_3_panel.pdf")
fig.savefig("../2_results/fig_3_panel.svg", format='svg')
plt.show()

### Manuscript Results - Part Two Sensitivity

1. Confidence Bounds
2. Covariance Structure 
3. Random Pertubation 

### Confidence Bounds

In [ ]:
import gsa_moe as gsa
from index_weights import dimension_weights
from plot_save import plot_sensitivity

n = 8192
batch_size = 10
cpu_cores = 150

# Define dimensions and their respective columns
dimensions = {
        'THEME1': ['HOCOB', 'POVTY', 'NOHIG', 'NOHEA'],
        'THEME2': ['AGE65', 'AGE17', 'DISBL', 'SNGPH', 'LANGU'],
        'THEME3': ['MINRTY'],
        'THEME4': ['MUUNS', 'MOHOM', 'CROWD', 'NOVEH']
    }

# denomination moe list
moe_list = ['estimate', 'lower_bound', 'upper_bound']

# normalization list
aggregation_list = ['arithmetic', 'geometric']

# normalization list
normalization_list = ['pct', 'z_score', 'min_max'] 

# weights list
weights_list = dimension_weights()

# Define dimensions and their respective columns
parameters = {
    'normalization': normalization_list,
    'aggregation': aggregation_list,
    'moe': moe_list,
    'weights': weights_list
}

# conduct global sensitivity analysis
sa_results = gsa.get_sensitivity(dimensions.copy(), parameters.copy(), n, 'SA', 500, cpu_cores, batch_size, 'nrq', 'z_score', 'confidence_bounds')

fig = plot_sensitivity(sa_results)
output_base = "../2_results/gitignore/0_new_bounds_bz_pzm_N8192"
fig.savefig(f"{output_base}.tiff", format='tiff', dpi=600)  
fig.savefig(f"{output_base}.pdf", format='pdf')             
fig.savefig(f"{output_base}.svg", format='svg') 

### Covariance Structure

In [ ]:
import gsa_moe as gsa
from index_weights import dimension_weights
from plot_save import plot_sensitivity

n = 8192
batch_size = 10
cpu_cores = 150

# Define dimensions and their respective columns
dimensions = {
        'THEME1': ['HOCOB', 'POVTY', 'NOHIG', 'NOHEA'],
        'THEME2': ['AGE65', 'AGE17', 'DISBL', 'SNGPH', 'LANGU'],
        'THEME3': ['MINRTY'],
        'THEME4': ['MUUNS', 'MOHOM', 'CROWD', 'NOVEH']
    }

# denomination moe list
moe_list = ['Estimate'] + [f'RepVar_{i}' for i in range(1, 81)]

# normalization list
aggregation_list = ['arithmetic', 'geometric']

# normalization list
normalization_list = ['pct', 'z_score', 'min_max'] 

# weights list
weights_list = dimension_weights()

# Define dimensions and their respective columns
parameters = {
    'normalization': normalization_list,
    'aggregation': aggregation_list,
    'moe': moe_list,
    'weights': weights_list
}

# conduct global sensitivity analysis
sa_results = gsa.get_sensitivity(dimensions.copy(), parameters.copy(), n, 'SA', 500, cpu_cores, batch_size, 'nrq', 'z_score', 'covariance_no_treat')

fig = plot_sensitivity(sa_results)
output_base = "../2_results/gitignore/0_new_cov_bz_pzm_N8192"
fig.savefig(f"{output_base}.tiff", format='tiff', dpi=600)   
fig.savefig(f"{output_base}.pdf", format='pdf')             
fig.savefig(f"{output_base}.svg", format='svg')

### Random Pertubation

In [ ]:
import gsa_moe as gsa
from index_weights import dimension_weights
from plot_save import plot_sensitivity

n = 8192
batch_size = 10
cpu_cores = 150

# Define dimensions and their respective columns
dimensions = {
        'THEME1': ['HOCOB', 'POVTY', 'NOHIG', 'NOHEA'],
        'THEME2': ['AGE65', 'AGE17', 'DISBL', 'SNGPH', 'LANGU'],
        'THEME3': ['MINRTY'],
        'THEME4': ['MUUNS', 'MOHOM', 'CROWD', 'NOVEH']
    }

# denomination moe list
moe_list = ['Estimate'] + [f'RepVar_{i}' for i in range(1, 81)]

# normalization list
aggregation_list = ['arithmetic', 'geometric']

# normalization list
normalization_list = ['pct', 'z_score', 'min_max'] 

# weights list
weights_list = dimension_weights()

# Define dimensions and their respective columns
parameters = {
    'normalization': normalization_list,
    'aggregation': aggregation_list,
    'moe': moe_list,
    'weights': weights_list
}

# conduct global sensitivity analysis
sa_results = gsa.get_sensitivity(dimensions.copy(), parameters.copy(), n, 'SA', 500, cpu_cores, batch_size, 'nrq', 'z_score', 'random_no_treat')

fig = plot_sensitivity(sa_results)
output_base = "../2_results/gitignore/0_new_rand_bz_pzm_N8192"
fig.savefig(f"{output_base}.tiff", format='tiff', dpi=600)   
fig.savefig(f"{output_base}.pdf", format='pdf')             
fig.savefig(f"{output_base}.svg", format='svg')